In [65]:
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

from haystack_integrations.components.generators.ollama import OllamaGenerator

document_store = InMemoryDocumentStore()
document_store.write_documents(
    [
        Document(content="Super Mario was an important politician"),
        Document(content="Mario owns several castles and uses them to conduct important political business"),
        Document(
            content="Super Mario was a successful military leader who fought off several invasion attempts by "
            "his arch rival - Bowser"
        ),
        Document(
            content="Sheikh Hasina Wazed[a] (the dictator) (born 28 September 1947) is a Bangladeshi politician "
            "and the tenth prime minister of Bangladesh from June 1996 to July 2001 and again serving "
            "since January 2009. She is the daughter of Sheikh Mujibur Rahman, the founding father and "
            "first president of Bangladesh. Having served for a combined total of over 20 years, she is "
            "the longest serving prime minister in the history of Bangladesh. As of 4 August 2024, she is "
            "the world's longest-serving female head of government.[2]"
        )
    ]
)

template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

pipe = Pipeline()
pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store, top_k=1))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component(
    "llm", 
    OllamaGenerator(
        model="gemma2:2b", 
        url="http://localhost:11434/api/generate",
        generation_kwargs={"temperature": 0.2}
    )
)
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")
query = "What is the age of Seikh hasina?"

response = pipe.run(
    {   "prompt_builder": {"query": query}, 
        "retriever": {"query": query}
    },
    include_outputs_from="prompt_builder"
)

# print(response["llm"]["replies"][0])

In [66]:
print(response['llm']['replies'][0])

Here's how to find Sheikh Hasina's age:

1. **Find her birthdate:** The text states she was born on September 28, 1947.
2. **Calculate the current year:**  The question asks for her age as of August 4, 2024.
3. **Subtract her birth year from the current year:** 2024 - 1947 = 77

**Therefore, Sheikh Hasina's age is 77.** 



In [53]:
print(response['prompt_builder']['prompt'])


Given only the following information, answer the question.
Ignore your own knowledge.

Context:

    Sheikh Hasina Wazed[a] (the dictator) (born 28 September 1947) is a Bangladeshi politician and the tenth prime minister of Bangladesh from June 1996 to July 2001 and again serving since January 2009. She is the daughter of Sheikh Mujibur Rahman, the founding father and first president of Bangladesh. Having served for a combined total of over 20 years, she is the longest serving prime minister in the history of Bangladesh. As of 4 August 2024, she is the world's longest-serving female head of government.[2]


Question: Who is Seikh hasina??


In [6]:
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

messages = [
    ChatMessage.from_user("What's Natural Language Processing?"),
    ChatMessage.from_system(
        "Natural Language Processing (NLP) is a field of computer science and artificial "
        "intelligence concerned with the interaction between computers and human language"
    ),
    ChatMessage.from_user("How do I get started?"),
]

client = OllamaChatGenerator(model="phi3:mini", timeout=45, url="http://localhost:11434/api/chat")
response = client.run(messages, generation_kwargs={"temperature": 0.2})
print(response["replies"][0].content)


In [67]:
help(client.run)

Help on method run in module haystack_integrations.components.generators.ollama.chat.chat_generator:

run(messages: List[haystack.dataclasses.chat_message.ChatMessage], generation_kwargs: Optional[Dict[str, Any]] = None) method of haystack_integrations.components.generators.ollama.chat.chat_generator.OllamaChatGenerator instance
    Runs an Ollama Model on a given chat history.

    :param messages:
        A list of ChatMessage instances representing the input messages.
    :param generation_kwargs:
        Optional arguments to pass to the Ollama generation endpoint, such as temperature,
        top_p, etc. See the
        [Ollama docs](https://github.com/jmorganca/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values).
    :param streaming_callback:
        A callback function that will be called with each response chunk in streaming mode.
    :returns: A dictionary with the following keys:
        - `replies`: The responses from the model



In [43]:
from haystack import Document
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()
documents = [Document(content="There are over 7,000 languages spoken around the world today."),
			       Document(content="Elephants have been observed to behave in a way that indicates a high level of self-awareness, such as recognizing themselves in mirrors."),
			       Document(content="In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the phenomenon of bioluminescent waves.")]

document_store.write_documents(documents= documents + [Document(content=f"nothing_{id}") for id in range(10)])

retriever = InMemoryBM25Retriever(document_store=document_store, top_k=1)
retriever.run(query="How many languages are spoken around the world today?")

{'documents': [Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', score: 14.802461901763598)]}

In [63]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

messages = [
    ChatMessage.from_system("Always respond in German even if some input data is in other languages."),
    ChatMessage.from_user("What's Natural Language Processing? Be brief."),
    ChatMessage.from_assistant("Sorry, I do not know German. Here is answer in English:\n")
]

generator = OllamaChatGenerator(
    model="phi3:mini",
    url = "http://localhost:11434/api/chat",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.,
    }
)

# chat_generator = OpenAIChatGenerator(model="gpt-3.5-turbo")
print(generator.run(messages=messages))

{'replies': [ChatMessage(content='\nNatural Language Processing (NLP) ist ein Teilgebiet der künstlichen Intelligenz und des maschinellen Lernens, das sich mit dem Verständnis, der Analyse und Generierung menschlicher Sprache befasst.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'phi3:mini', 'created_at': '2024-08-05T00:21:16.699691Z', 'done_reason': 'stop', 'done': True, 'total_duration': 2828576875, 'load_duration': 10948666, 'prompt_eval_count': 57, 'prompt_eval_duration': 324602000, 'eval_count': 54, 'eval_duration': 2456960000})]}


In [64]:
print(messages)

[ChatMessage(content='Always respond in German even if some input data is in other languages.', role=<ChatRole.SYSTEM: 'system'>, name=None, meta={}), ChatMessage(content="What's Natural Language Processing? Be brief.", role=<ChatRole.USER: 'user'>, name=None, meta={}), ChatMessage(content='Sorry, I do not know German. Here is answer in English:\n', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={})]


In [69]:
from haystack.components.builders import PromptBuilder

something = PromptBuilder(messages)

TypeError: Can't compile non template nodes